In [ ]:
#import necessary libraries
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, BertForSequenceClassification, BertTokenizer
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.functional import softmax
from google.cloud import storage 
import io

In [ ]:
# Declare global variables
GCP_KEY = "/Users/lu31635/Desktop/AC215/ac215.json"
GCP_DATA_BUCKET = 'data-lnt'
GCP_SOURCE_FILENAME = 'processed/initial_labels.csv'
GCP_MODELS_BUCKET = 'models-lnt'

HIGH_CONFIDENCE_THRESHOLD = 0.9
TEST_SIZE = 0.2
NUMBER_EPOCHS = 3
RANDOM_STATE = 215
ADAM_LEARNING_RATE = 1e-5
ADAM_BATCH_SIZE = 32
LABEL_BATCH_SIZE = 64

MODEL_DIR_FINETUNE = 'fine_tune_label'
MODEL_DIR_BERT = 'bert_label'

SENTIMENT_LABELS = ["Negative", "Neutral", "Positive"]
GCP_OUTPUT_PATH = 'processed/labeled_final.csv'


In [ ]:
##create GCP Client
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = GCP_KEY
source_filename = GCP_SOURCE_FILENAME
storage_client = storage.Client()
data_bucket = storage_client.bucket(GCP_DATA_BUCKET)
models_bucket = storage_client.bucket(GCP_MODELS_BUCKET)
blob = data_bucket.blob(source_filename)
content = blob.download_as_text()
#Sanity check
print(content[:20])

In [ ]:
def get_model(model_directory):
    """
    Input: model_name (name of desired BERT model)
    Output: tokenizer, model

    >>> get_model("cardiffnlp/twitter-xlm-roberta-base-sentiment")
    tokenizer(model_name), model(model_name)
    """
    tokenizer = AutoTokenizer.from_pretrained(model_directory)
    model = AutoModelForSequenceClassification.from_pretrained(model_directory)

    return tokenizer, model

In [ ]:
def get_high_confidence_df(df, threshold=HIGH_CONFIDENCE_THRESHOLD):
    """
    Filters an input dataframe to only include samples with label confidence above a defined threshold

    Input: dataframe with initial labels, desired confidence threshold
    Output: dataframe including only high confidence examples above specified threshold

    >>> get_high_confidence_df(df, 0.9)
    high_confidence_df
    """
    return df[(df['negative_score'] > threshold) |
    (df['neutral_score'] > threshold) |
    (df['positive_score'] > threshold)]

In [ ]:
def tokenize(dataframe):
    """
    Returns tokenized text given a dataframe

    Input: Pandas dataframe (assumes text column = 'text')
    Output: tokenized text

    >>> tokenize(df)
    tokenized_texts
    """
    text_samples = dataframe['text'].tolist()
    tokenized_texts = tokenizer(text_samples, padding=True, truncation=True, return_tensors="pt")

    return tokenized_texts

In [ ]:
def get_datasets(df, tokenizer, test_size=TEST_SIZE):
    """
    Returns training and validation datasets given a dataframe and tokenizer

    Input: panadas dataframe, tokenizer, 
    Output: tokenized text

    >>> tokenize(df)
    tokenized_texts
    """
    # Define training and valid dataframes
    train_df, valid_df = train_test_split(df, test_size=test_size, random_state=RANDOM_STATE)

    # Tokenize the training data
    train_encodings = tokenizer(train_df['text'].tolist(), truncation=True, padding=True, return_tensors='pt')
    train_labels = torch.tensor(train_df['label'].tolist())

    # Tokenize the validation data
    valid_encodings = tokenizer(valid_df['text'].tolist(), truncation=True, padding=True, return_tensors='pt')
    valid_labels = torch.tensor(valid_df['label'].tolist())

    # Create DataLoader objects
    train_dataset = TensorDataset(train_encodings.input_ids, train_encodings.attention_mask, train_labels)
    valid_dataset = TensorDataset(valid_encodings.input_ids, valid_encodings.attention_mask, valid_labels)

    return train_dataset, valid_dataset

In [ ]:
def train_bert(model, train_dataset, valid_dataset, epochs=NUMBER_EPOCHS):
    """
    Fine tunes the pretrained BERT model based on the high confidence samples

    Input: BERT model, training dataset, validation dataset, number of epochs
    Output: None (Prints epoch progress)

    >>> train_bert(high_confidence_df, train_data, valid_data, epochs=4)
    Epoch 2/4: Validation Loss: 12.3452, Validation Accuracy: 0.8362
    """
    
    # Define device for compuation
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Train loop
    optimizer = AdamW(model.parameters(), lr=ADAM_LEARNING_RATE)
    train_loader = DataLoader(train_dataset, batch_size=ADAM_BATCH_SIZE, shuffle=True)
    for epoch in range(epochs):
        model.train()
        for batch in train_loader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

    # Validation loop
    valid_loader = DataLoader(valid_dataset, batch_size=ADAM_BATCH_SIZE)
    model.eval()
    with torch.no_grad():
        total_loss = 0.0
        correct = 0
        total = 0
        for batch in valid_loader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = torch.nn.functional.cross_entropy(logits, labels)
            total_loss += loss.item()

            _, predicted = torch.max(logits, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        accuracy = correct / total
        average_loss = total_loss / len(valid_loader)

        print(f'Epoch {epoch + 1}/{epochs}: Validation Loss: {average_loss:.4f}, Validation Accuracy: {accuracy:.4f}')

In [ ]:
def save_model(output_directory, model, tokenizer):
    """
    Saves the final fine tuned model and tokenizer to GCP models bucket

    Input: GCP output directory, model, tokenizer
    Output: None
    """

    output_dir = models_bucket.blob(output_directory)
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

In [ ]:
def label(model, df, batch_size=LABEL_BATCH_SIZE):
    """
    Uses the fine-tuned model to evaluate the full initial dataset. New labels are added to the dataframe
    based on the label provided by the fine-tuned model.

    Input: fine-tuned model, full dataframe
    Output: None
    """

    # Define tokenized text
    tokenized_texts = tokenize(df)
    
    #get input IDs and attention mask from tokenized text
    input_ids = tokenized_texts['input_ids']
    attention_mask = tokenized_texts['attention_mask']
    
    #define dataset from input IDs and attention mask
    dataset = TensorDataset(input_ids, attention_mask)

    #define batch size and create DataLoader
    batch_size = batch_size
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    #set model to evaluation mode
    model.eval()

    #create empty list to store labels for entire dataset
    labels = []
    
    # Iterate through the DataLoader and use the model for labeling
    batch = 0
    num_batches = -(-len(input_ids) // batch_size)
    for batch_input_ids, batch_attention_mask in dataloader:
        batch += 1
        print(f"Batch {batch} of {num_batches}")

        with torch.no_grad():
            batch_labels = model(batch_input_ids, attention_mask=batch_attention_mask)
        
        #append batch labels to dataset label list
        labels.append(batch_labels)
        
    #concatenate all labels
    labels = torch.cat(labels, dim=0)

    #define output logits
    logits = labels.logits

    #map the predicted labels to sentiment categories
    sentiment_labels = SENTIMENT_LABELS
    predicted_labels = [sentiment_labels[label] for label in torch.argmax(logits, dim=1).tolist()]

    #add the predicted labels to the DataFrame
    df['label_fine_tune'] = predicted_labels.tolist()

In [ ]:
def save_dataset(df, outfilepath):
    """
    Saves the labeled dataframe to GCP data bucket
    
    Input: Pandas dataframe, GCP file path
    Output: None

    >>> save_dataset(dataframe, 'filepath'):
    returns None
    """

    out_file = data_bucket.blob(outfilepath)
    df.to_csv(out_file, index=False)
    out_file.upload_from_filename(outfilepath)

In [ ]:
# Load the pre-trained model and tokenizer
try: 
    model_directory = models_bucket.blob(MODEL_DIR_FINETUNE)
except:
    model_directory = models_bucket.blob(MODEL_DIR_BERT)

tokenizer, model = get_model(model_directory)

In [ ]:
# Import labeled dataset 
initial_df = pd.read_csv(io.StringIO(content))

# Filter high-confidence examples based on predicted sentiment scores
high_confidence_df = get_high_confidence_df(initial_df, HIGH_CONFIDENCE_THRESHOLD)
high_confidence_df.head(5)

In [ ]:
# Get training and validation datasets 
train_data, valid_data = get_datasets(high_confidence_df, tokenizer, TEST_SIZE)

In [ ]:
# Fine-tune the BERT model
train_bert(model, train_data, valid_data, epochs=NUMBER_EPOCHS)

In [ ]:
# Save the fine-tuned model to GCP
save_model('fine_tune_label', model, tokenizer)

In [ ]:
# Label the full dataset using the fine-tuned model
label(model, initial_df)

In [ ]:
# Export final dataframe to GCP
save_dataset(initial_df, GCP_OUTPUT_PATH)